In [1]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 17:55:20] [setup] RAM Tracking...
[codecarbon INFO @ 17:55:20] [setup] CPU Tracking...
[codecarbon WARNING @ 17:55:23] No CPU tracking mode found. Falling back on CPU constant mode. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 17:55:24] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 17:55:24] [setup] GPU Tracking...
[codecarbon INFO @ 17:55:24] No GPU found.
[codecarbon INFO @ 17:55:24] >>> Tracker's metadata:
[codecarbon INFO @ 17:55:24]   Platform system: macOS-10.16-x86_64-i386-64bit
[codecarbon INFO @ 17:55:24]   Python version: 3.9.12
[codecarbon INFO @ 17:55:24]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 17:55:24]   Available RAM : 8.000 GB
[codecarbon INFO @ 17:55:24]   CPU count: 8
[codecarbon INFO @ 17:55:24]   CPU model: Apple M1
[codecarbon INFO @ 17:55:24]   GPU count: None
[codecarbon INFO @ 17:55:24]   GPU model: None
[codecarbon INFO @ 17:55:28] Saving emissions data to

In [2]:
import pandas as pd
import numpy as np

# Cargar el dataset (ajusta la ruta si es necesario)
df = pd.read_csv("../training.csv")

# Vista rápida de las primeras filas
df.head()


,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,...,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653,s
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,...,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584,b
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,...,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389,b
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,5.446378,b
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,6.245333,b


In [3]:
print(f"Número de ejemplos en el archivo: {len(df)}")

Número de ejemplos en el archivo: 250000


In [4]:
print("Columnas del dataset:")
print(df.columns.tolist())


Columnas del dataset:
['EventId', 'DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt', 'Weight', 'Label']


In [5]:
# Separar target y features válidos
X = df.drop(columns=['EventId', 'Weight', 'Label'])
y = df['Label']
y = y.map({'s': 1, 'b': 0})
# Confirmamos shapes
print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")

import numpy as np
from sklearn.impute import SimpleImputer

# Reemplazar -999.0 por NaN
X.replace(-999.0, np.nan, inplace=True)

# Imputar con la media
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Volver a DataFrame con las mismas columnas
X = pd.DataFrame(X_imputed, columns=X.columns)



Shape de X: (250000, 30)
Shape de y: (250000,)


In [6]:
from sklearn.model_selection import train_test_split

# División: 80% entrenamiento, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Tamaño X_train: {X_train.shape}")
print(f"Tamaño X_test: {X_test.shape}")


Tamaño X_train: (200000, 30)
Tamaño X_test: (50000, 30)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Definir el grid de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 5],
    'criterion': ['gini', 'entropy']
}

# Instanciar el clasificador base
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Búsqueda con validación cruzada
grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    verbose=2
)

# Entrenar
grid_rf.fit(X_train, y_train)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


[codecarbon INFO @ 17:55:43] Energy consumed for RAM : 0.000013 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:55:43] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:55:43] 0.000033 kWh of electricity used since the beginning.


[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  13.5s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  12.6s


[codecarbon INFO @ 17:55:58] Energy consumed for RAM : 0.000025 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:55:58] Energy consumed for all CPUs : 0.000042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:55:58] 0.000067 kWh of electricity used since the beginning.


[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  12.7s


...

[CV] END criterion=entropy, max_depth=20, min_samples_leaf=5, min_samples_split=10, n_estimators=200; total time=  28.8s


[codecarbon INFO @ 19:05:21] Energy consumed for RAM : 0.003483 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 19:05:21] Energy consumed for all CPUs : 0.005810 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:05:21] 0.009292 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:05:36] Energy consumed for RAM : 0.003495 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 19:05:36] Energy consumed for all CPUs : 0.005830 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:05:36] 0.009326 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:05:36] 0.000386 g.CO2eq/s mean an estimation of 12.176140622793369 kg.CO2eq/year


[CV] END criterion=entropy, max_depth=20, min_samples_leaf=5, min_samples_split=10, n_estimators=200; total time=  28.3s


[codecarbon INFO @ 19:05:51] Energy consumed for RAM : 0.003508 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 19:05:51] Energy consumed for all CPUs : 0.005851 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:05:51] 0.009359 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:06:06] Energy consumed for RAM : 0.003520 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 19:06:06] Energy consumed for all CPUs : 0.005872 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:06:06] 0.009392 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:06:21] Energy consumed for RAM : 0.003533 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 19:06:21] Energy consumed for all CPUs : 0.005893 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:06:21] 0.009426 kWh of electricity used since the beginning.


GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 5],
                         'min_samples_split': [2, 10],
                         'n_estimators': [100, 200]},
             scoring='f1', verbose=2)

In [8]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# Guardar el mejor modelo
best_rf = grid_rf.best_estimator_

# Predecir en test
y_pred = best_rf.predict(X_test)
y_proba = best_rf.predict_proba(X_test)[:, 1]

# Métricas
print("🔍 Mejores parámetros encontrados:")
print(grid_rf.best_params_)
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification report:")
print(classification_report(y_test, y_pred, digits=4))
print("\nAUC-ROC:", roc_auc_score(y_test, y_proba))


🔍 Mejores parámetros encontrados:
{'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 200}

Accuracy: 0.83882

Classification report:
              precision    recall  f1-score   support

           0     0.8584    0.9040    0.8806     32867
           1     0.7949    0.7138    0.7522     17133

    accuracy                         0.8388     50000
   macro avg     0.8266    0.8089    0.8164     50000
weighted avg     0.8366    0.8388    0.8366     50000


AUC-ROC: 0.9060980717861513


In [9]:
emissions = tracker.stop()

[codecarbon INFO @ 19:06:22] Energy consumed for RAM : 0.003534 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 19:06:22] Energy consumed for all CPUs : 0.005894 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:06:22] 0.009428 kWh of electricity used since the beginning.
